#Top five reinsurance companies financial data analysis

This colab notebook serves as a machine for ingesting and transforming publicly available financial data from the top five reinsurance companies. The main aim is to clean each data set, bring all the data together and finally visualise the numbers to conduct a peer analysis. At the same time the data must be stored in a format which enables an easy and automated update for future financial data releases.

Although the files are available on each companies investor relations website, I downloaded it and stored it in a [github repository](https://github.com/lukasbelka/Reinsurers-financial-data-analysis) together with this notebook. The repository also contains [the links](https://github.com/lukasbelka/Reinsurers-financial-data-analysis/blob/main/raw_financial_data/readme.md) to the companies websites.

## Data Ingestion, exploration and cleaning

The first step is to import the relevant packages and load the financial data from the repository.

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

<IPython.core.display.Javascript object>

In [2]:
url = ('https://github.com/lukasbelka/Reinsurers-financial-data-analysis/blob/main/raw_financial_data/HNR/HNR_AR2021_XLS_en.xlsx?raw=true')
xls_21 = pd.read_excel(url, sheet_name=None)

# Let's see which sheets the excel file contains
print(xls_21.keys())    

dict_keys(['Table of content', 'Key figures', 'Remuneration Executive Board', 'Remuneration Supervisory Board', 'Consolidated balance sheet', 'Consolid. statement of income', 'Consolid. comprehensive income', "Consolid. shareholders' equity", 'Consolid. cash flow statement', 'Segment report - balance sheet', 'Segment report - income statem.', 'Investments by currencies', 'Gross written premium', 'Investment income', 'Net loss reserve P&C', 'Gains&losses on investments'])


As a first step I will load the income statements sheet and have a quick look at the raw data.

In [13]:

# The data frame from the income statements sheet will be called finally 'is_hr' after the cleanup.

is_hr_2021_raw = pd.read_excel(url, sheet_name='Consolid. statement of income', skiprows=2, header=1)
(is_hr_2021_raw.info())
(is_hr_2021_raw.head())
(is_hr_2021_raw)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   In EUR thousand      37 non-null     object 
 1   Notes                20 non-null     float64
 2   1.1. - 31.12.2021    34 non-null     float64
 3   1.1. - 31.12.2020¹   34 non-null     float64
dtypes: float64(3), object(1)
memory usage: 1.3+ KB


In EUR thousand  Notes  \
0                               Gross written premium   7.10   
1                               Ceded written premium    NaN   
2                    Change in gross unearned premium    NaN   
3                    Change in ceded unearned premium    NaN   
4                                  Net premium earned    NaN   
5                          Ordinary investment income   7.20   
6   Profit/loss from investments in associated com...   7.20   
7            Realised gains and losses on investments   7.20   
8       Change in fair value of financial instruments   7.20   
9   Total depreciation, impairments and appreciati...   7.20   
10                          Other investment expenses   7.20   
11   Net income from investments under own management    NaN   
12  Income/expense on funds withheld and contract ...   7.20   
13                              Net investment income    NaN   
14                             Other technical income   7.30   
15                                     Total revenues    NaN   
16                         Claims and claims expenses   7.30   
17                         Change in benefit reserves   7.30   
18  Commission and brokerage, change in deferred a...   7.30   
19                            Other acquisition costs    NaN   
20                            Administrative expenses   7.30   
21                           Total technical expenses    NaN   
22                                       Other income   7.40   
23                                     Other expenses   7.40   
24                              Other income/expenses   7.40   
25                       Operating profit/loss (EBIT)    NaN   
26                                    Financing costs   6.12   
27                            Net income before taxes    NaN   
28                                              Taxes   7.50   
29                                         Net income    NaN   
30                                            thereof    NaN   
31        Non-controlling interest in profit and loss   6.14   
32                                   Group net income    NaN   
33                        Earnings per share (in EUR)   8.50   
34                           Basic earnings per share    NaN   
35                         Diluted earnings per share    NaN   
36                                                NaN    NaN   
37                                                NaN    NaN   
38                       ¹ Restated pursuant to IAS 8    NaN   

    1.1. - 31.12.2021  1.1. - 31.12.2020¹   
0         27762314.00          24770342.00  
1          2905054.00           2442720.00  
2          -737631.00          -1028172.00  
3            24023.00             61345.00  
4         24143652.00          21360795.00  
5          1555591.00           1240420.00  
6            35743.00             88129.00  
7           281026.00            329610.00  
8            36114.00             63971.00  
9            87665.00            129393.00  
10          146047.00            129034.00  
11         1674762.00           1463703.00  
12          268250.00            221765.00  
13         1943012.00           1685468.00  
14             114.00                15.00  
15        26086778.00          23046278.00  
16        18617725.00          16782658.00  
17         -298645.00           -103487.00  
18         5788582.00           5111753.00  
19            4767.00              4466.00  
20          510707.00            478182.00  
21        24623136.00          22273572.00  
22          774816.00            823391.00  
23          503631.00            382022.00  
24          271185.00            441369.00  
25         1734827.00           1214075.00  
26           83037.00             90204.00  
27         1651790.00           1123871.00  
28          351569.00            205086.00  
29         1300221.00            918785.00  
30                NaN                  NaN  
31           68887.00             357

We have the following observations:
- Numbers in EUR thousand
- Four columns in the consolidated income statement.
  - The first one shows the names of all income statement positions
  - The second one seems to contain notes references
  - The third and fourth hold the figures for the financial years 2020 and 2021.
- We have some NaN (**N**ot **a** **N**umber) values.

Let's deal with the data as follows:
- Skip the first two rows of the sheet to obtain the third one as the header (done already while creating the data frame)
- Clean the data by dropping all NaN-rows in order to get rid of empty cells
- Delete the second column with notes references
- Include a column with the company name (which enables us to identify the positions (rows) after combining the data from all reinsurers

Here we go!

In [12]:
is_hr_2021_drop_c = is_hr_2021_raw.drop(columns='Notes')
is_hr_2021_drop_c.drop(is_hr_2021_drop_c.tail(3).index, inplace=True)
is_hr_final = is_hr_2021_drop_c.rename(columns={"1.1. - 31.12.2021": "2021", "1.1. - 31.12.2020¹ ": "2020"})

is_hr_final

In EUR thousand         2021  \
0                               Gross written premium  27762314.00   
1                               Ceded written premium   2905054.00   
2                    Change in gross unearned premium   -737631.00   
3                    Change in ceded unearned premium     24023.00   
4                                  Net premium earned  24143652.00   
5                          Ordinary investment income   1555591.00   
6   Profit/loss from investments in associated com...     35743.00   
7            Realised gains and losses on investments    281026.00   
8       Change in fair value of financial instruments     36114.00   
9   Total depreciation, impairments and appreciati...     87665.00   
10                          Other investment expenses    146047.00   
11   Net income from investments under own management   1674762.00   
12  Income/expense on funds withheld and contract ...    268250.00   
13                              Net investment income   1943012.00   
14                             Other technical income       114.00   
15                                     Total revenues  26086778.00   
16                         Claims and claims expenses  18617725.00   
17                         Change in benefit reserves   -298645.00   
18  Commission and brokerage, change in deferred a...   5788582.00   
19                            Other acquisition costs      4767.00   
20                            Administrative expenses    510707.00   
21                           Total technical expenses  24623136.00   
22                                       Other income    774816.00   
23                                     Other expenses    503631.00   
24                              Other income/expenses    271185.00   
25                       Operating profit/loss (EBIT)   1734827.00   
26                                    Financing costs     83037.00   
27                            Net income before taxes   1651790.00   
28                                              Taxes    351569.00   
29                                         Net income   1300221.00   
30                                            thereof          NaN   
31        Non-controlling interest in profit and loss     68887.00   
32                                   Group net income   1231334.00   
33                        Earnings per share (in EUR)          NaN   
34                           Basic earnings per share        10.21   
35                         Diluted earnings per share        10.21   

           2020  
0   24770342.00  
1    2442720.00  
2   -1028172.00  
3      61345.00  
4   21360795.00  
5    1240420.00  
6      88129.00  
7     329610.00  
8      63971.00  
9     129393.00  
10    129034.00  
11   1463703.00  
12    221765.00  
13   1685468.00  
14        15.00  
15  23046278.00  
16  16782658.00  
17   -103487.00  
18   5111753.00  
19      4466.00  
20    478182.00  
21  22273572.00  
22    823391.00  
23    382022.00  
24    441369.00  
25   1214075.00  
26     90204.00  
27   1123871.00  
28    205086.00  
29    918785.00  
30          NaN  
31     35712.00  
32    883073.00  
33          NaN  
34         7.32  
35         7.32